In [1]:
import numpy as np
import simpy
from scipy.stats import expon

In [17]:
def arrivals(env,server,S,G,stop):
    while True:
        dt = expon.rvs(scale = 1)
        arrival = env.timeout(dt)
        yield arrival
        if env.now < 10:
            S.append(env.now)
            env.process(service(env,server,S,G,stop))
            # stop processing arrival after 10
        elif len(server.queue) + len(server.users) == 0:
            stop.succeed()
            yield stop

        
        
def service(env,server,S,G,stop):
    rqt = server.request() #you will be added to a list. When you yield the request
    yield rqt #triggered
    service_time = 0.8
    service = env.timeout(service_time)
    yield service
    G.append(env.now)
    server.release(rqt)
        
    
    
            
def simulate():
    S = []
    G = []
    env = simpy.Environment()
    stop  =env.event()
    server = simpy.Resource(env,capacity = 1)
    env.process(arrivals(env,server,S,G,stop))
    env.run(until = stop)
    return max(G[-1] - 10,0)

X = [simulate() for i in range(1000)]
print(np.mean(X))

1.1739730948133411
